In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

from modules.mail import send_checks_xq_mail

import sys
from config.variables import user

In [2]:
training_code = 'EQP'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['tipo'] = pd.to_numeric(training['RegistrationNumber'], errors='coerce').notnull()
training = training[training['tipo']]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao', 'tipo']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
1234,00000021,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,69",2020-12-06 11:27:15
1157,00000022,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,91",2020-10-11 20:49:08
1103,00000024,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,93",2020-12-08 17:38:07
925,00000026,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,20",2020-12-11 22:45:17
903,00000027,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,36",2020-08-05 17:25:04
...,...,...,...,...,...
215,00044787,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,63",2020-10-30 13:36:43
780,00044789,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,32",2020-10-19 19:04:09
1081,00044795,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,09",2020-10-24 18:31:13
1135,00045913,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"7,70",2020-12-23 12:58:27


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)
rosters

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


,cif,code,act_type,airline,fleet,tail,leg_type,fr,to,dep,arr,pairingId
0,00010103,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 06:05:00,6173246.0
1,00006285,FP,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
2,00022301,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 12:00:00,6099354.0
3,00036105,FANI,A,NaN,NaN,NaN,NaN,BSB,BSB,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
4,00013384,SOBREAVIS,A,NaN,NaN,NaN,NaN,POA,POA,2020-01-01 06:00:00,2020-01-01 14:15:00,6173917.0
...,...,...,...,...,...,...,...,...,...,...,...,...
202411,00010475,FANI,A,NaN,NaN,NaN,NaN,GRU,GRU,2021-02-01 05:50:00,2021-02-02 05:50:00,NaN
202412,00035321,G3 1703,L,G3,73M,NaN,J,REC,CGH,2021-02-01 05:50:00,2021-02-01 09:10:00,6720235.0
202413,00023217,G3 1703,L,G3,73M,NaN,J,REC,CGH,2021-02-01 05:50:00,2021-02-01 09:10:00,6720278.0
202414,00000085,FPT,A,NaN,NaN,NaN,NaN,CGH,CGH,2021-02-01 05:55:00,2021-02-02 05:55:00,NaN


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
79226,00008078,INSS
79236,00001215,INSS
79244,00002976,INSS
79255,00025077,INSS
79257,00031556,INSS
...,...,...
80657,00044628,INSS
80658,00024329,INSS
80662,00029447,INSS
80663,00032709,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
79230,00008652,LIC-SREM
79232,00000873,LIC-SREM
79235,00001154,LIC-SREM
79238,00002163,LIC-SREM
79240,00002744,LIC-SREM
...,...,...
80641,00046263,LIC-SREM
80642,00046270,LIC-SREM
80643,00046277,LIC-SREM
80644,00046310,LIC-SREM


In [9]:
ensino = rosters.copy()
ensino = ensino[ensino['code'].isin(funcs.training_filter(training_code))]
ensino = ensino[ensino['code'] != 'XQ-SIMU']
ensino = ensino.sort_values(by=['cif','dep'])
ensino = ensino[['cif', 'code', 'dep']]
ensino.columns = ['cif', 'ensino', 'data_ensino']
ensino = ensino.drop_duplicates(subset = ['cif'], keep = 'last')
ensino

,cif,ensino,data_ensino
76436,00000020,C-ENS-737,2020-03-11 11:15:00
144417,00000021,C-737-ON,2020-12-23 11:15:00
79043,00000022,C-737-ON,2020-10-14 11:30:00
65143,00000024,C-737-ON,2020-12-11 11:15:00
65178,00000026,C-737-ON,2020-12-11 11:15:00
...,...,...,...
96986,99999910,C-737-ON,2020-12-16 11:15:00
138661,99999912,C-ENS-737,2020-02-19 11:15:00
191055,99999916,C-737-ON,2020-12-30 11:15:00
31701,99999917,C-737-ON,2020-11-06 11:30:00


In [10]:
xq_simu = rosters.copy()
xq_simu = xq_simu[xq_simu['code'] == 'XQ-SIMU']
xq_simu = xq_simu.sort_values(by=['cif','dep'])
xq_simu = xq_simu[['cif', 'code', 'dep']]
xq_simu = xq_simu.drop_duplicates(subset = ['cif'], keep = 'last')
xq_simu.columns = ['cif', 'xq', 'data_xq']
xq_simu

,cif,xq,data_xq
108318,00000020,XQ-SIMU,2020-03-15 16:50:00
35305,00000021,XQ-SIMU,2021-01-06 13:35:00
43978,00000022,XQ-SIMU,2020-11-08 12:40:00
102602,00000024,XQ-SIMU,2021-01-16 12:40:00
76750,00000027,XQ-SIMU,2020-08-15 08:30:00
...,...,...,...
147079,99999018,XQ-SIMU,2020-02-20 12:40:00
147921,99999910,XQ-SIMU,2021-01-23 13:35:00
146268,99999912,XQ-SIMU,2020-10-25 13:35:00
147100,99999916,XQ-SIMU,2020-02-20 12:40:00


In [11]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
6,00000610,EQP,2021-06-30
46,00001615,EQP,2021-09-30
59,00004743,EQP,2021-05-31
60,00006016,EQP,2021-09-30
62,00003158,EQP,2021-07-31
...,...,...,...
47405,00042970,EQP,2021-09-30
47406,00042971,EQP,2021-09-30
50033,00044695,EQP,2021-04-30
50498,00044775,EQP,2021-04-30


In [12]:
# Merge dos dados
data_all = pd.merge(crew, checks, on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(ensino, on='cif', how='left')
data_all = data_all.merge(xq_simu, on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,ensino,data_ensino,xq,data_xq,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,EQP,2021-03-31,NaN,NaN,NaN,NaT,C-ENS-737,2020-03-11 11:15:00,XQ-SIMU,2020-03-15 16:50:00,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,EQP,2021-02-28,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,69",2020-12-06 11:27:15,C-737-ON,2020-12-23 11:15:00,XQ-SIMU,2021-01-06 13:35:00,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,EQP,2021-11-30,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,91",2020-10-11 20:49:08,C-737-ON,2020-10-14 11:30:00,XQ-SIMU,2020-11-08 12:40:00,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,93",2020-12-08 17:38:07,C-737-ON,2020-12-11 11:15:00,XQ-SIMU,2021-01-16 12:40:00,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,20",2020-12-11 22:45:17,C-737-ON,2020-12-11 11:15:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,EQP,2021-02-28,NaN,NaN,NaN,NaT,C-737-ON,2020-12-30 11:15:00,XQ-SIMU,2020-02-20 12:40:00,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,EQP,2021-11-30,NaN,NaN,NaN,NaT,C-737-ON,2020-11-06 11:30:00,XQ-SIMU,2020-11-12 12:40:00,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,EQP,2021-02-28,NaN,NaN,NaN,NaT,C-737-ON,2020-12-23 11:15:00,NaN,NaN,NaN,NaN


In [13]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

# Ajuste de data do treinamento online
validation['data_ensino'] = validation['data_ensino'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_ensino'] = pd.to_datetime(validation['data_ensino'], format='%Y-%m-%d')
validation['data_ensino'] = validation['data_ensino'].dt.floor('D')

# Ajuste de data do treinamento online
validation['data_xq'] = validation['data_xq'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_xq'] = pd.to_datetime(validation['data_xq'], format='%Y-%m-%d')
validation['data_xq'] = validation['data_xq'].dt.floor('D')

# Data da conclusao ON LINE
validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')


validation['presencial_depois_online'] = validation['data_xq'] > validation['data_ensino']
validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation = validation[validation['data_xq'] < (pd.to_datetime(dt.date(dt.now())))]

validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['presencial_depois_online'])

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,ensino,data_ensino,xq,data_xq,inss,lnr,presencial_depois_online,atualizar
19,00000073,ANDRE MARTINS,CMT,IBX,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,57",2020-11-04,C-737-ON,2020-11-12,XQ-SIMU,2020-01-09,INSS,NaN,False,False
23,00000226,SAMIR,CMT,IR,GRU,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,46",2020-12-16,C-737-ON,2020-12-16,XQ-SIMU,2020-01-25,NaN,NaN,False,False
30,00000330,POSI,CMT,IBX,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,40",2020-12-09,C-737-ON,2020-12-10,XQ-SIMU,2021-01-07,NaN,NaN,True,True
35,00000338,RAFAEL,CMT,IRX,POA,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Em andamento,"0,00",2035-12-31,C-737-ON,2021-01-06,XQ-SIMU,2021-01-11,NaN,NaN,True,False
85,00000644,TITO,CMT,IRX,BSB,EQP,2021-01-31,NaN,NaN,NaN,2035-12-31,C-737-ON,2020-12-11,XQ-SIMU,2020-01-16,INSS,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,00035461,ANDRESSA IONGBLOD,COP,NaN,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,51",2020-12-05,C-737-ON,2020-12-05,XQ-SIMU,2021-01-04,NaN,NaN,True,True
1600,00041638,THIAGO FISCHER,COP,NaN,GIG,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,46",2020-12-03,C-737-ON,2020-12-05,XQ-SIMU,2021-01-10,NaN,NaN,True,True
1608,00041647,CLAUDIO PINHEIRO,CMT,NaN,GIG,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,90",2020-12-03,C-737-ON,2020-12-04,XQ-SIMU,2021-01-10,NaN,NaN,True,True
1764,00044013,TOSTES,COP,NaN,GIG,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,33",2020-12-02,C-737-ON,2020-12-05,XQ-SIMU,2021-01-06,NaN,NaN,True,True


In [14]:
validation.to_csv(f'./data/output/{training_code}.csv', index=False, sep=';', encoding='utf-8-sig')

In [15]:
# Arquivo de import
# fill table
df = validation[validation['atualizar']]

df

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,ensino,data_ensino,xq,data_xq,inss,lnr,presencial_depois_online,atualizar
30,00000330,POSI,CMT,IBX,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,40",2020-12-09,C-737-ON,2020-12-10,XQ-SIMU,2021-01-07,NaN,NaN,True,True
139,00001181,PEINADO,CMT,IRX,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,50",2020-12-16,C-737-ON,2020-12-16,XQ-SIMU,2021-01-07,NaN,NaN,True,True
248,00002346,ADONIDES,CMT,IR,BSB,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,60",2020-12-07,C-737-ON,2020-12-08,XQ-SIMU,2021-01-11,NaN,NaN,True,True
262,00002443,FERNANDO HAUPT,CMT,NaN,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,59",2020-11-17,C-737-ON,2020-11-20,XQ-SIMU,2021-01-07,NaN,NaN,True,True
302,00002852,MARTINI,CMT,NaN,GRU,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,03",2020-12-10,C-737-ON,2020-12-10,XQ-SIMU,2021-01-12,NaN,NaN,True,True
338,00003075,ALYSSON GUSTAVO,CMT,IR,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,50",2020-12-08,C-737-ON,2020-12-10,XQ-SIMU,2021-01-08,NaN,NaN,True,True
364,00003166,FERNANDA PRIETO,CMT,IR,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,29",2020-12-10,C-737-ON,2020-12-10,XQ-SIMU,2021-01-10,NaN,NaN,True,True
394,00003453,SCARIN FILHO,CMT,NaN,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,41",2020-12-05,C-737-ON,2020-12-10,XQ-SIMU,2021-01-07,NaN,NaN,True,True
424,00003775,STOLZE,CMT,NaN,CGH,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"8,59",2020-12-18,C-737-ON,2020-12-18,XQ-SIMU,2021-01-06,NaN,NaN,True,True
429,00003781,SADI,CMT,IR,GRU,EQP,2021-01-31,Tripulaçao Tecnica - Ensino Periódico B737 (IF...,Concluído,"9,20",2020-12-07,C-737-ON,2020-12-08,XQ-SIMU,2021-01-04,NaN,NaN,True,True


In [16]:
df_ = df[['cif', 'checkTypeCode', 'data_conclusao']]

# fill table

data = df_.copy()

data['update'] = 'U'
data['id'] = df_['cif'].astype(str).str.zfill(8)
data['type'] = 'XQ'
data['issue'] = df_['data_conclusao'].dt.strftime('%Y%m%d') + '0000'
data['expiry'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['extended'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['failed'] = 'N'
data['do_not_plan_before'] = (df_['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['zero'] = '0'
data['remark'] = ''
data['check_comment'] = ''
data['confirmed'] = 'Y'
data['paper_recieved'] = 'Y'

# remove fields from original file
cols = [
    'update',
    'cif',
    'type',
    'issue',
    'expiry',
    'extended',
    'failed',
    'do_not_plan_before',
    'zero',
    'remark',
    'check_comment',
    'confirmed',
    'paper_recieved'  
]

data = data[cols].copy()

data.to_csv('./data/output/crm_import_xq.dat', sep='|', header=None, index=False, line_terminator='\r\n')

data

,update,cif,type,issue,expiry,extended,failed,do_not_plan_before,zero,remark,check_comment,confirmed,paper_recieved
30,U,00000330,XQ,202012090000,202112090000,202112090000,N,202112090000,0,,,Y,Y
139,U,00001181,XQ,202012160000,202112160000,202112160000,N,202112160000,0,,,Y,Y
248,U,00002346,XQ,202012070000,202112070000,202112070000,N,202112070000,0,,,Y,Y
262,U,00002443,XQ,202011170000,202111170000,202111170000,N,202111170000,0,,,Y,Y
302,U,00002852,XQ,202012100000,202112100000,202112100000,N,202112100000,0,,,Y,Y
338,U,00003075,XQ,202012080000,202112080000,202112080000,N,202112080000,0,,,Y,Y
364,U,00003166,XQ,202012100000,202112100000,202112100000,N,202112100000,0,,,Y,Y
394,U,00003453,XQ,202012050000,202112050000,202112050000,N,202112050000,0,,,Y,Y
424,U,00003775,XQ,202012180000,202112180000,202112180000,N,202112180000,0,,,Y,Y
429,U,00003781,XQ,202012070000,202112070000,202112070000,N,202112070000,0,,,Y,Y


In [17]:
out_email = validation.copy()

out_email = out_email[~out_email['inss'].isin(['INSS'])]
out_email = out_email[~out_email['lnr'].isin(['LIC-SREM'])]

out_email = out_email[['cif',
                       'guerra',
                       'funcao',
                       'base',
                       'checkTypeCode',
                       'expiryDt', 
                       'status', 
                       'nota', 
                       'data_conclusao',
                       'ensino',
                       'data_ensino',
                       'xq',
                       'data_xq',
                       'atualizar']]

out_email['expiryDt'] = out_email['expiryDt'].dt.strftime('%d/%m/%Y')
out_email['data_conclusao'] = out_email['data_conclusao'].dt.strftime('%d/%m/%Y')
out_email['data_ensino'] = out_email['data_ensino'].dt.strftime('%d/%m/%Y')
out_email['data_xq'] = out_email['data_xq'].dt.strftime('%d/%m/%Y')
out_email['status'] = out_email['status'].fillna('')
out_email['nota'] = out_email['nota'].fillna('')

In [18]:
out_email

,cif,guerra,funcao,base,checkTypeCode,expiryDt,status,nota,data_conclusao,ensino,data_ensino,xq,data_xq,atualizar
23,00000226,SAMIR,CMT,GRU,EQP,31/01/2021,Concluído,"8,46",16/12/2020,C-737-ON,16/12/2020,XQ-SIMU,25/01/2020,False
30,00000330,POSI,CMT,CGH,EQP,31/01/2021,Concluído,"9,40",09/12/2020,C-737-ON,10/12/2020,XQ-SIMU,07/01/2021,True
35,00000338,RAFAEL,CMT,POA,EQP,31/01/2021,Em andamento,"0,00",31/12/2035,C-737-ON,06/01/2021,XQ-SIMU,11/01/2021,False
139,00001181,PEINADO,CMT,CGH,EQP,31/01/2021,Concluído,"9,50",16/12/2020,C-737-ON,16/12/2020,XQ-SIMU,07/01/2021,True
199,00001919,MAURICIO KELMANN,CMT,CGH,EQP,31/01/2021,Em andamento,"0,00",31/12/2035,C-737-ON,23/01/2021,XQ-SIMU,12/01/2020,False
230,00002126,LEONARDO SANTIAGO,CMT,BSB,EQP,31/01/2021,Concluído,"9,00",05/12/2020,C-737-ON,05/12/2020,XQ-SIMU,14/01/2020,False
248,00002346,ADONIDES,CMT,BSB,EQP,31/01/2021,Concluído,"9,60",07/12/2020,C-737-ON,08/12/2020,XQ-SIMU,11/01/2021,True
262,00002443,FERNANDO HAUPT,CMT,CGH,EQP,31/01/2021,Concluído,"8,59",17/11/2020,C-737-ON,20/11/2020,XQ-SIMU,07/01/2021,True
278,00002581,RAMON,CMT,CGH,EQP,31/01/2021,Em andamento,"8,83",31/12/2035,C-737-ON,16/12/2020,XQ-SIMU,07/01/2021,False
302,00002852,MARTINI,CMT,GRU,EQP,31/01/2021,Concluído,"8,03",10/12/2020,C-737-ON,10/12/2020,XQ-SIMU,12/01/2021,True


In [19]:
send_checks_xq_mail(out_email, user['mail'], user['password'], user['emails'])